# Example: Get, inspect, change, and put back a instrument method

Here outlines how to interact with instrument methods and module methods.

- Finding and getting a instrument method.
- Inspecting what types of module methods are in the instrument method.
- Inspecting the instrument parameters in the method, including column temperature and gradient table.
- Changing instrument parameters and posting the new method.

# Finding and getting a instrument method

We first find the list of all instrument methods, select one, and get that from Empower:

In [3]:
import pprint

from OptiHPLCHandler import EmpowerHandler

# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address="XXXXXX")

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = method_list[0]  # Select the first method
    print(method_name)
    full_method = handler.GetInstrumentMethod(method_name)

Get call to endpoint project/methods?methodTypes=MethodSetMethod could be slow, timeout is set to 20 seconds
@BSM_PDA_Template


# Inspecting the instrument method

We can now look into the module methods in the instrument method. 

In [3]:
pp = pprint.PrettyPrinter(indent=2)

print(full_method)
print(f"Valve positions: {full_method.module_method_list[-1].valve_position}")
# Printing the valve position for the solvent manager module method in the list.
# Notice that we do not need to know the tag name to print the valve position.
print("\n\nStart of gradient table:\n")
pp.pprint(full_method.gradient_table[0:2])
# Printing the first two entries gradient table.
# Notice that we do not need to know the tag names to print the gradient table
print("\n\nDetector settings:")
print(
    full_method.module_method_list[0]["ChannelA"]
)  # Printing the settings for a single channel where we know the tag name (ChannelA)

EmpowerInstrumentMethod with 4 module methods of types EmpowerModuleMethod, EmpowerModuleMethod, ColumnManagerMethod, BSMMethod
Valve positions: ['A2', 'B2']


Start of gradient table:

[ { 'CompositionA': '95.0',
    'CompositionB': '5.0',
    'Curve': 'Initial',
    'Flow': '0.900',
    'Time': 'Initial'},
  { 'CompositionA': '20.0',
    'CompositionB': '80.0',
    'Curve': '6',
    'Flow': '0.900',
    'Time': '2.00'}]


Detector settings:

    <Description />
    <Wavelength>280</Wavelength>
    <DataRate>SingleDataRate_20A</DataRate>
    <DataMode>SingleMode_1A</DataMode>
    <FilterType>Filter_2</FilterType>
    <TimeConstant>0.1</TimeConstant>
    <RatioMinimum>0.0001</RatioMinimum>
    <AutoZeroWavelength>Az_3</AutoZeroWavelength>
    <AutoZeroInjectStart>true</AutoZeroInjectStart>
    <AutoZeroEventOrKey>true</AutoZeroEventOrKey>
  


Note that most of the methods are simply EmpowerModuleMethods. That is the generic
type for module methods that aren't specifically accounted for. At the moment, that
is every type of method expect:
 - Solvent manager methods (BSMMethod and QSMMethod), which have the special properties `valve_position` and `gradient_table`.
 - Column oven methods (SampleManagerMethod), which have the special property `column_temperature`.

All of the special properties can be accessed as set from the `EmpowerInstrumentMethod`.
Getting `EmpowerInstrumentMethod.column_temperature` will produce an error if the
instrument method controls several column ovens (e.g. one in a sample manager and one in
a column manager), and the column ovens have different temperatures. Setting 
`EmpowerInstrumentMethod.column_temperature` will set the temperature for all column
ovens if there are any, and produce an error otherwise.

# Changing instrument parameters and posting the new method.

We can also change the values of the instrument method, and post the changed method back
to Empower, so we can use it to analyse samples.

Remember to give the method a new name before posting it to Empower. OptiHPLCHandler
does not allow changing methods in Empower, only creating new ones.

In [4]:
gradient_table = full_method.gradient_table  # Get the gradient table
for step in gradient_table:
    step["Flow"] = 0.5  # Set the flow to 0.5 mL/min for all steps
gradient_table[1]["Time"] = 5 / 3  # Set the time for the second step to 5/3 minutes.
# Notice the warning that 1.666666667 minutes is rounded to 1.667, since Empower will
# misinterpret values with too many decimals.
full_method.gradient_table = (
    gradient_table  # Set the gradient table to the updated gradient table
)
full_method.valve_position = [
    "A2",
    "B1",
]  # Set the valve position to A2 and B1. You can also set only one of the valves.
full_method.method_name = "New method name"  # Set the method name
with handler:
    handler.PostInstrumentMethod(full_method)  # Post the updated method to Empower

Rounding 1.6666666666666667 to 1.667, as Empower only accepts 3 decimals.


HTTPError: HTTP error 409 with message 'Project - Login:  Invalid User/Password.' and ID 8dbe7490ee3986b